# 0050 回測模型
### 2003~2018
> 每年投入固定資金，上漲x%賣i股,下跌y%買j股

In [1]:
# 讀取csv(資料來源XQ匯入excel調整)
#['時間(yyyy/mm/dd)', '開盤價', '最高價', '最低價', '收盤價', '漲跌幅(%)', '成交量']
import csv
temp=[]
temp_all=[[] for i in range(19)]
with open('0050.csv') as cf:
    rows = csv.reader(cf)
    for row in rows:
        temp.append(row)
        
# str->float
for i in range(len(temp)):
    for j in range(len(temp[i])):
        hh = temp[i][j]
        if type(hh)==str:
            try:
                temp[i][j]=float(hh)
            except:
                pass
            
## 以年分割資料(2000年以後)
itertemp = iter(temp)
next(itertemp)
for i in itertemp:
    year = int(i[0][0:4])-2000
    temp_all[year].append(i)
    
#除權息和金額    
dividend={
'2018/7/23' : 0.7,
'2018/1/29' : 2.2,
'2017/7/31' : 0.7,
'2017/2/8' : 1.7,
'2016/7/28' : 0.85,
'2015/10/26' : 2,
'2014/10/24' : 1.55,
'2013/10/24' : 1.3,
'2012/10/24' : 1.85,
'2011/10/26' : 1.95,
'2010/10/25' : 2.2,
'2009/10/23' : 1,
'2008/10/24' : 2,
'2007/10/24' : 2.5,
'2006/10/26' : 4,
'2005/5/19' : 1.85
}

In [2]:
# 測試股利
import doctest
def test_dividend():
    """
    >>> '2005/5/19' in dividend.keys()
    True
    >>> temp[1075][0]  in dividend.keys()
    True
    """
doctest.testmod()

TestResults(failed=0, attempted=2)

In [3]:
def findbest(start_money,upper,lower,upper_sell,lower_buy,temp):
    
    ##初始化金額、股數、flag
    money=start_money
    stock = 0
    flag=False
    
    # 如果傳的是年就合併所有資料
    if len(temp) < 30:
        flag=True
        test=[]
        for i in temp:
            test.extend(i)
        temp=test

    
    ## 從有漲跌幅的開始到結束
    count=1
    for row in temp[2:-1]:
        
        #每年投入start_money
        count+=1
        if count%250==0:
            money+=start_money
            
        
        #除權息
        if row[0] in dividend.keys():
            money+=stock*dividend[row[0]]
            
        
        #漲賣,賺錢
        if row[5]>upper and stock>upper_sell:
            stock-=upper_sell
            money+=row[4]*upper_sell
            if flag:
                print("{}賣出\t剩餘:{:.0f}元\t剩餘:{}股".format(row[0],money,stock))
                   
        #跌買,花錢
        elif row[5]<-1*lower and money>lower_buy*row[4]:
            stock+=lower_buy
            money-=row[4]*lower_buy
            if flag:
                print("{}買進\t剩餘:{:.0f}元\t剩餘:{}股".format(row[0],money,stock))
        
    total = int(money+row[4]*stock)
    
    #年回測
    if flag:
        print("-------------------------------------")
        print("如果漲{}%,賣{}股,跌{}%,買{}股".format(upper,upper_sell,lower,lower_buy))
        print("結:total={},money={},stock={}".format(total,money,stock))
        print("淨利:%d萬\t總投入%d萬"%((total-count/250*start_money)/10000,(count/250*start_money)/10000))
        

    #全部回測，而且只取賺的,return 總投入(萬),總金額(萬),現金(萬),股數  
    elif total > 0:
        return [count/250*start_money/10000,total/10000,money/10000,stock]
       

# 回測0.5%~3%,0~3000股

In [5]:
print("Find best parameter")
print("===================")
##跑回側
# x=每期投入幾萬元(range)
# i*10=upperbound
# j*10=lowerbound
# k*100=buy
# l*100=sell
data=[]
for x in range(10,31,1):
    money=x*10000
    earn_max=[0,0,0,0]
    best_parameter=[0,0,0,0]
    for i in range(5,31,5):
        for j in range(5,31,5):
            for k in range(0,26,5):
                for l in range(0,26,5):
                    try:
                        earn=findbest(money,i/10,j/10,k*100,l*100,temp)
                        if earn_max[1]<earn[1]:
                            earn_max=earn.copy()
                            best_parameter=[i/10,j/10,k*100,l*100]
                    except:
                        pass
                        
    data.append([x,best_parameter])
    
    print("如果漲{0[0]}%,賣{0[2]}股,跌{0[1]}%,買{0[3]}股".format(best_parameter))
    print("結:total={0[1]}萬元,money={0[2]}萬元,stock={0[3]}".format(earn_max))
    print("起始資金{}萬元，總投入{}萬元".format(x,int(earn_max[0])))
    print('---------------------------------------------')
                   

Find best parameter
如果漲1.5%,賣2000股,跌0.5%,買2500股
結:total=341.9685萬元,money=1.971萬元,stock=40500
起始資金10萬元，總投入150萬元
---------------------------------------------
如果漲1.5%,賣1500股,跌0.5%,買2000股
結:total=372.099萬元,money=11.114萬元,stock=43000
起始資金11萬元，總投入165萬元
---------------------------------------------
如果漲2.0%,賣2500股,跌0.5%,買2000股
結:total=404.8695萬元,money=6.107萬元,stock=47500
起始資金12萬元，總投入180萬元
---------------------------------------------
如果漲2.0%,賣2000股,跌0.5%,買2500股
結:total=440.662萬元,money=16.7145萬元,stock=50500
起始資金13萬元，總投入195萬元
---------------------------------------------
如果漲2.0%,賣2500股,跌0.5%,買2000股
結:total=471.4775萬元,money=5.555萬元,stock=55500
起始資金14萬元，總投入210萬元
---------------------------------------------
如果漲2.0%,賣2500股,跌0.5%,買2000股
結:total=499.4785萬元,money=4.1735萬元,stock=59000
起始資金15萬元，總投入225萬元
---------------------------------------------
如果漲3.0%,賣2500股,跌0.5%,買1000股
結:total=533.4805萬元,money=8.793萬元,stock=62500
起始資金16萬元，總投入240萬元
---------------------------------------------
如果漲3.0%,賣2500股,跌2.0

In [6]:
## 統計
from collections import Counter
test=[str(i[1]) for i in data]
counter=Counter(test)
counter

Counter({'[1.5, 0.5, 1500, 2000]': 1,
         '[1.5, 0.5, 2000, 2500]': 2,
         '[2.0, 0.5, 2000, 1500]': 2,
         '[2.0, 0.5, 2000, 2000]': 3,
         '[2.0, 0.5, 2000, 2500]': 1,
         '[2.0, 0.5, 2500, 2000]': 5,
         '[2.0, 0.5, 2500, 2500]': 5,
         '[3.0, 0.5, 2500, 1000]': 1,
         '[3.0, 2.0, 2500, 2000]': 1})

In [7]:
dict_count={}
for i in range(1,7):
    for j in range(1,7):
        dict_count[str([i*0.5,j*0.5])]=0

for i in range(len(data)):
    dict_count[str(data[i][1][0:2])]+=1
dict_count

{'[0.5, 0.5]': 0,
 '[0.5, 1.0]': 0,
 '[0.5, 1.5]': 0,
 '[0.5, 2.0]': 0,
 '[0.5, 2.5]': 0,
 '[0.5, 3.0]': 0,
 '[1.0, 0.5]': 0,
 '[1.0, 1.0]': 0,
 '[1.0, 1.5]': 0,
 '[1.0, 2.0]': 0,
 '[1.0, 2.5]': 0,
 '[1.0, 3.0]': 0,
 '[1.5, 0.5]': 3,
 '[1.5, 1.0]': 0,
 '[1.5, 1.5]': 0,
 '[1.5, 2.0]': 0,
 '[1.5, 2.5]': 0,
 '[1.5, 3.0]': 0,
 '[2.0, 0.5]': 16,
 '[2.0, 1.0]': 0,
 '[2.0, 1.5]': 0,
 '[2.0, 2.0]': 0,
 '[2.0, 2.5]': 0,
 '[2.0, 3.0]': 0,
 '[2.5, 0.5]': 0,
 '[2.5, 1.0]': 0,
 '[2.5, 1.5]': 0,
 '[2.5, 2.0]': 0,
 '[2.5, 2.5]': 0,
 '[2.5, 3.0]': 0,
 '[3.0, 0.5]': 1,
 '[3.0, 1.0]': 0,
 '[3.0, 1.5]': 0,
 '[3.0, 2.0]': 1,
 '[3.0, 2.5]': 0,
 '[3.0, 3.0]': 0}

## 每年20萬,[2, 0.5, 2500, 2000]

In [8]:
#年回測
findbest(200000,2,0.5,2500,2000,temp_all[0:19])


2003/7/10買進	剩餘:119900元	剩餘:2000股
2003/7/15買進	剩餘:39520元	剩餘:4000股
2003/7/24賣出	剩餘:142595元	剩餘:1500股
2003/7/29買進	剩餘:60555元	剩餘:3500股
2003/8/13賣出	剩餘:166680元	剩餘:1000股
2003/8/26買進	剩餘:79760元	剩餘:3000股
2003/8/29賣出	剩餘:192060元	剩餘:500股
2003/9/3買進	剩餘:102060元	剩餘:2500股
2003/9/4買進	剩餘:13040元	剩餘:4500股
2003/9/8賣出	剩餘:127915元	剩餘:2000股
2003/9/9買進	剩餘:36515元	剩餘:4000股
2003/10/2賣出	剩餘:149140元	剩餘:1500股
2003/10/8買進	剩餘:57140元	剩餘:3500股
2003/10/13賣出	剩餘:175115元	剩餘:1000股
2003/10/23買進	剩餘:80915元	剩餘:3000股
2003/12/1賣出	剩餘:196415元	剩餘:500股
2003/12/9買進	剩餘:104415元	剩餘:2500股
2003/12/10買進	剩餘:12935元	剩餘:4500股
2004/1/2賣出	剩餘:131185元	剩餘:2000股
2004/1/29買進	剩餘:33985元	剩餘:4000股
2004/3/1賣出	剩餘:163860元	剩餘:1500股
2004/3/5買進	剩餘:58860元	剩餘:3500股
2004/3/18賣出	剩餘:185610元	剩餘:1000股
2004/3/22買進	剩餘:90750元	剩餘:3000股
2004/3/29賣出	剩餘:213500元	剩餘:500股
2004/3/31買進	剩餘:115520元	剩餘:2500股
2004/4/1買進	剩餘:18120元	剩餘:4500股
2004/4/5賣出	剩餘:143120元	剩餘:2000股
2004/4/9買進	剩餘:43520元	剩餘:4000股
2004/4/12賣出	剩餘:170895元	剩餘:1500股
2004/4/15買進	剩餘:70095元	剩餘:3500股
2004/5/4賣出	剩餘:187595元	剩餘:1000股
2